In [43]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#  Packages & Imports

In [99]:
%load_ext autoreload
%aimport data_prep
%run data_prep.py
%aimport modeling
%run modeling.py
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load prepared Data

In [88]:
#Load Datasets
data = pd.read_csv("./FE_Final/train_data_prepared.csv")
target = pd.read_csv("./original_datasets/training_set_labels.csv")
target=target[["status_group"]]

In [89]:
data = data.drop(columns=["subvillage", "ward"])
data = onehot_encode(data)

# of features before: 36
# of features after: 199


In [90]:
data.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,Flag_population,Flag_longitude,...,extraction_type_other,extraction_type_submersible,extraction_type_swn 80,payment_never pay,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown
0,6000.000000,1390,34.938093,-9.856322,0,11,5,109,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.000000,1399,34.698766,-2.147466,0,20,2,280,0,0,...,0,0,0,1,0,0,0,0,0,0
2,25.000000,686,37.460664,-3.821329,0,0,4,250,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.000000,263,38.486161,-11.155298,0,0,63,58,0,0,...,0,1,0,1,0,0,0,0,0,0
4,317.650385,0,31.130847,-1.825359,0,18,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0


# Baseline Model (Random Forest)

In [94]:
X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.8)

rf = RFC(n_estimators = 1000, random_state = random_seed, n_jobs=-1)

rf.fit(X_train, y_train)

rf_score = rf.score(X_test,y_test)

print('Accuracy:', rf_score)

Accuracy: 0.8086700336700336


# Multi-class logReg

In [92]:
clf = LogisticRegression(multi_class="ovr").fit(data, target)
clf_Score = clf.score(X_test,y_test)
print('Accuracy:', clf_Score)

Accuracy: 0.5398148148148149


# XGBoost w/ GS

In [ ]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300],
              'random_state':[random_seed]
}

model = xgb.XGBRegressor()

to_int = {'functional':1,'non functional':2,'functional needs repair':3}
to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

numerical_target = target.status_group.replace(to_int)
numerical_test = y_test.status_group.replace(to_int)

import re

data = data.rename(lambda x: re.sub("[\]\(\]\)\, ]","",x), axis='columns')

xg = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = 2,
                    n_jobs = -1,
                    scoring = 'r2',
                    verbose=True)
xg.fit(data, numerical_target)
xg.score(X_test,numerical_test)

Fitting 2 folds for each of 288 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
